### Now lets create Chatbot with multiple tools

In [2]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [3]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = 'llama3.2')

In [4]:
from datetime import datetime

def date():
    """ Returns today's date, month and year
    args: none
    return: today's date, month and year which is a datetime object
    """
    return datetime.now().date()

def dt():
    """ Returns the current time
    args: none
    return: current time which is a datetime object
    """
    return datetime.now().time()

In [15]:
tools = [date,dt]

In [16]:
llm_with_tools = llm.bind_tools(tools)

In [22]:
def llm_tool(state:State):
    return {'messages':[llm_with_tools.invoke(state['messages'])]}

In [24]:
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)

builder.add_node("llm_calling_tools",llm_tool)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"llm_calling_tools")
builder.add_conditional_edges('llm_calling_tools', tools_condition)
builder.add_edge("tools",END)

builder_graph = builder.compile()

In [25]:
from pprint import pprint

In [26]:
response = builder_graph.invoke({'messages':"What is the current date?"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the current date?
================================== Ai Message ==================================
Tool Calls:
  date (ece82fa0-6907-4f4e-9d2f-f7af3e909adf)
 Call ID: ece82fa0-6907-4f4e-9d2f-f7af3e909adf
  Args:
================================= Tool Message =================================
Name: date

2025-09-28


In [27]:
response = builder_graph.invoke({'messages':"What is the current time?"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the current time?
================================== Ai Message ==================================
Tool Calls:
  dt (9c99569a-8ac8-44ac-89a8-81b857de2e48)
 Call ID: 9c99569a-8ac8-44ac-89a8-81b857de2e48
  Args:
================================= Tool Message =================================
Name: dt

15:49:24.317456


***Let's perform an Experiment***

### Redirecting the output of Tool Node back to LLM

In [29]:
builder1 = StateGraph(State)

builder1.add_node("llm_calling_tools",llm_tool)
builder1.add_node("tools",ToolNode(tools))

builder1.add_edge(START,"llm_calling_tools")
builder1.add_conditional_edges('llm_calling_tools', tools_condition)
builder1.add_edge("tools","llm_calling_tools")
builder1.add_edge("llm_calling_tools",END)

builder_graph1 = builder1.compile()

In [30]:
response = builder_graph1.invoke({'messages':"What is the current time?"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the current time?
================================== Ai Message ==================================
Tool Calls:
  dt (848591a4-5b4f-46eb-86f3-ace5f524fdeb)
 Call ID: 848591a4-5b4f-46eb-86f3-ace5f524fdeb
  Args:
================================= Tool Message =================================
Name: dt

15:51:48.789852
================================== Ai Message ==================================

The current time is 3:51 PM.


In [31]:
response = builder_graph1.invoke({'messages':"What is the today's date?"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the today's date?
================================== Ai Message ==================================
Tool Calls:
  date (99ceea78-72fc-4441-97df-df26fad74230)
 Call ID: 99ceea78-72fc-4441-97df-df26fad74230
  Args:
================================= Tool Message =================================
Name: date

2025-09-28
================================== Ai Message ==================================

The current date is September 28, 2025.


In [32]:
response = builder_graph1.invoke({'messages':"I want to know today's date as well as time"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know today's date as well as time
================================== Ai Message ==================================
Tool Calls:
  date (20901d9c-87b2-4d6a-989a-c46e4837a376)
 Call ID: 20901d9c-87b2-4d6a-989a-c46e4837a376
  Args:
  dt (44eb2a13-66f9-4256-8384-4d3a3aecf34b)
 Call ID: 44eb2a13-66f9-4256-8384-4d3a3aecf34b
  Args:
================================= Tool Message =================================
Name: date

2025-09-28
================================= Tool Message =================================
Name: dt

15:55:02.687346
================================== Ai Message ==================================

The current date is September 28, 2025, and the current time is 3:55 PM.
